In [49]:
import pandas as pd
import numpy as np
import scipy.stats as stats

#import dataframe
df = pd.read_excel('/Users/rayhowanski/Documents/Data Science/Bioinformatics/20201117_12009_Data_Volcano plots.xlsx',
                  sheet_name='Blood Vs LN',
                  skiprows=(1,2))
#clean up gene column
df.rename(columns={ df.columns[0]: "Gene" }, inplace = True)
df['Gene'] = df['Gene'].astype(str).str.replace('-mRNA', '')

stat = df.copy()

In [50]:
#extract blood data
blood = df.iloc[:, 1:4]
blood['mean'] = blood.mean(axis=1)
blood['std']=blood.iloc[:,0:3].std(axis=1)
LN = df.iloc[:,4:7]
LN['mean'] = LN.mean(axis=1)
LN['std']=LN.iloc[:,0:3].std(axis=1)


In [51]:
#calculate log2 of fold change, just to visualize data
df['FC']=blood['mean'] / LN['mean']
log2_FC = df['FC'].to_numpy()
df['log2_FC']=np.log2(log2_FC)

In [53]:
#take the second to last string in column name between '-' to call column Blood, LN, etc
stat.columns = stat.columns.str.split('-').str[-2]
stat.rename(columns={ stat.columns[0]: "Gene" }, inplace = True)

In [54]:
#melt df based on sample type variable
stat_melt = pd.melt(stat, id_vars=['Gene'], value_vars=['Blood','LN'])

In [67]:
#statistical analysis
def get_ttest(x,y,sided=1):
    return stats.ttest_ind(x, y, equal_var=False).pvalue/sided

df =stat_melt

groupby = 'Gene'
test_control = 'variable'
effect = 'value'

a,b = df[test_control].unique()

df_pval = df.groupby([groupby,test_control])\
            [effect].agg(['size','mean']).unstack(test_control)

df_pval.columns = [f'group{a}_size',f'group{b}_size',
                   f'group{a}_mean',f'group{b}_mean']

df_pval['pvalue'] = df.groupby(groupby).apply(lambda dfx: get_ttest(
    dfx.loc[dfx[test_control] == a, effect],
    dfx.loc[dfx[test_control] == b, effect]))

In [68]:
df_pval['FC'] = df_pval['groupBlood_mean'] / df_pval['groupLN_mean']
log2_FC = df_pval['FC'].to_numpy()
df_pval['log2_FC'] = np.log2(log2_FC)

In [69]:
log10_p = df_pval['pvalue'].to_numpy()
df_pval['log10_p'] = (np.log10(log10_p))*(-1)

In [70]:
vol = df_pval.copy().reset_index()
#vol.to_excel('/Volumes/LabData/Utilities/Blood_v_LN_data_input.xlsx')

In [88]:
vol['Label']=''

In [89]:
vol['Gene_Regulation']='Not Significant'
vol.loc[vol.log2_FC > 1.5, 'Gene_Regulation'] = 'Up Regulated'
vol.loc[vol.log2_FC < -1.5, 'Gene_Regulation'] = 'Down Regulated'

In [90]:
vol.loc[vol.log10_p < 1.5, 'Gene_Regulation'] = 'Not Significant'

In [91]:
vol.loc[vol.Gene_Regulation == 'Up Regulated', 'Label'] = vol['Gene']

In [92]:
vol.loc[vol.Gene_Regulation == 'Down Regulated', 'Label'] = vol['Gene']

In [93]:
import plotly.express as px
fig = px.scatter(vol, x="log2_FC", y="log10_p", hover_data=['Gene'], color="Gene_Regulation", text='Label',color_discrete_sequence=["black", "green", "red"]
                ,title="Nanostring mRNA Analysis of Blood Vs Lymph Node")
fig.update_traces(textposition='top left',textfont_size=10)
fig.show()

In [94]:
fig.write_html('/Volumes/LabData/Utilities/Blood_v_LN_Volcano.html', auto_open = False)

In [95]:
vol

,Gene,groupBlood_size,groupLN_size,groupBlood_mean,groupLN_mean,pvalue,FC,log2_FC,log10_p,Label,Gene_Regulation
0,A2M,3,3,31.293317,48.370664,0.370317,0.646948,-0.628278,0.431426,,Not Significant
1,ACVR1C,3,3,109.596135,123.091499,0.596828,0.890363,-0.167534,0.224151,,Not Significant
2,ADAM12,3,3,31.395258,136.298632,0.150362,0.230342,-2.118153,0.822862,,Not Significant
3,ADGRE1,3,3,667.626877,144.732757,0.191322,4.612825,2.205651,0.718236,,Not Significant
4,ADM,3,3,332.618654,71.153039,0.153007,4.674694,2.224872,0.815289,,Not Significant
...,...,...,...,...,...,...,...,...,...,...,...
745,XCL1/2,3,3,326.303492,719.508213,0.160271,0.453509,-1.140797,0.795146,,Not Significant
746,ZAP70,3,3,2916.604747,3211.603012,0.423765,0.908146,-0.139004,0.372875,,Not Significant
747,ZC3H12A,3,3,339.740074,284.105670,0.516005,1.195823,0.258004,0.287346,,Not Significant
748,ZEB1,3,3,940.311393,1571.453121,0.056072,0.598371,-0.740889,1.251257,,Not Significant
